In [ ]:
ls

In [ ]:
cd ..

In [ ]:
!source venv/bin/activate

In [ ]:
from bs4 import BeautifulSoup
import re
from datasets import load_dataset
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

from rag_techniques.fusion_retrieval import FusionRag
from research.functions import get_natural_questions_sample, get_ms_marco_sample

In [ ]:
model = FusionRag(collection_name='natural_questions')
df = pd.read_csv('research/data/qa_dataset.csv', index_col=0)

In [ ]:
natural_questions_dataset = {
    "question": [],
    "answer": [],
    "contexts": [],
    "ground_truth": [],
}

for i, (question, answer) in tqdm(df.iterrows(), total=df.shape[0]):
    result = model(question)
    natural_questions_dataset["question"].append(question)
    natural_questions_dataset["answer"].append(result['answer'])
    natural_questions_dataset["contexts"].append(result['context'])
    if pd.isna(answer):
        answer = 'No answer'
    natural_questions_dataset["ground_truth"].append(answer)    

In [ ]:
new_df = pd.DataFrame(natural_questions_dataset)
new_df.to_csv('research/data/fusion_rag.csv')

In [ ]:
from src.answer_correctness import AnswerCorrectness

evaluator = AnswerCorrectness()
res = evaluator.get_correctness(natural_questions_dataset['question'], natural_questions_dataset['answer'], natural_questions_dataset['ground_truth'])
np.mean(res[1])

In [ ]:
from src.ragas import Evaluator
ragas = Evaluator()

In [ ]:
e = ragas.eval(natural_questions_dataset)